[Reference](https://cees-roele.medium.com/styling-complex-values-in-a-dataframe-7ebc5a17b3e7)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Create a 5x5 matrix of random numbers between [-0.3,0.7)
m = np.random.rand(5,5) - 0.3
df = pd.DataFrame(m)
df.style.background_gradient()

,0,1,2,3,4
0,0.622125,0.257838,0.135233,0.535091,0.227610
1,0.628711,-0.111232,0.229143,0.651015,0.016267
2,-0.127933,-0.217844,0.080531,0.008445,0.159393
3,0.428820,-0.086931,0.339639,0.112919,-0.043466
4,0.035853,-0.248090,0.467629,0.200647,0.149727


In [3]:
def style_negative(v, props=''):
    return props if v < 0 else None
df.style.applymap(style_negative, props='color:red;')

,0,1,2,3,4
0,0.622125,0.257838,0.135233,0.535091,0.227610
1,0.628711,-0.111232,0.229143,0.651015,0.016267
2,-0.127933,-0.217844,0.080531,0.008445,0.159393
3,0.428820,-0.086931,0.339639,0.112919,-0.043466
4,0.035853,-0.248090,0.467629,0.200647,0.149727


In [4]:
import re
# "A Visit from St. Nicholas" by Clement Clarke Moore
# https://www.poetryfoundation.org/poems/43171/a-visit-from-st-nicholas
poem = """
'Twas the night before Christmas, when all through the house
Not a creature was stirring, not even a mouse;
The stockings were hung by the chimney with care,
In hopes that St. Nicholas soon would be there;
"""
words = re.split(r'\W+', poem)
pairs = [(x, len(x)) for x in words]
# Create a 5x5 matrix of tuples: (word, length)
matrix = []
for i in range(0,5):
    matrix.append(pairs[5*i:5*(i+1)])
df2 = pd.DataFrame(matrix)

In [5]:
df2.style.format(lambda x: x[0])

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


In [6]:
def font_short(v, weight, color):
    """
    Mark short words
    
    Parameters
    ----------
    v: tuple of (word, length)
    weight: CSS term accepted as a font-weight value
    color: CSS term accepted as a color
    """
    return f"font-weight: {weight}; color: {color}" if v[1] < 4 else None
# We must apply both `format` and `applymap` to the DataFrame.style
styler = df2.style.format(lambda x: x[0])
styler.applymap(font_short, weight='bold', color='orange')

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


In [7]:
import matplotlib as mpl
def make_gradient(v, min_length, max_length, cmap='YlGn'):
    """
    Parameters
    ----------
    
    v: tuple of (word, length)
    min_length: int 
        minimum length of all words in the matrix
    max_length: int
        maximum length of all words in the matrix
    cmap: matplotlib color map, default value here is 'YlGn'
        
    Returns
    -------
    
    string:
        CSS setting a colour
              
    For Matplotlib colormaps:
    See: https://matplotlib.org/stable/tutorials/colors/colormaps.html
"""
    # normalize the word length as a fraction of the range 
    # between min_length and max_length
    rel_v = (v[1] - min_length) / (max_length - min_length)
    # define the colormap
    cmap = mpl.cm.get_cmap(cmap)
    # Get a colour out of the given colormap based on a value [0,1]
    rgba = cmap(rel_v)  
    # convert the colour to a hexadecimal string representation
    return f'background-color: {mpl.colors.rgb2hex(rgba)};'
# We must apply both `format` and `applymap` to the DataFrame.style
styler = df2.style.format(lambda x: x[0])
min_length = min([x[1] for x in pairs])
max_length = max([x[1] for x in pairs])
styler.applymap(lambda x: make_gradient(x, min_length, max_length))

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


In [8]:
# Create one dataframe with only words
df_words = df2.applymap(lambda x: x[0])
# .. and a second one with only lengths
df_lengths = df2.applymap(lambda x: x[1])
# Now use the lengths to apply the background gradient to the words
df_words.style.background_gradient(gmap=df_lengths, axis=None)

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


In [9]:
df_lengths.applymap(lambda x: f'length: {x}')

,0,1,2,3,4
0,length: 0,length: 4,length: 3,length: 5,length: 6
1,length: 9,length: 4,length: 3,length: 7,length: 3
2,length: 5,length: 3,length: 1,length: 8,length: 3
3,length: 8,length: 3,length: 4,length: 1,length: 5
4,length: 3,length: 9,length: 4,length: 4,length: 2


In [10]:
styler = df_words.style.background_gradient(gmap=df_lengths,     axis=None, cmap='YlOrRd')
styler.set_tooltips(
    df_lengths.applymap(lambda x: f'length: {x}'), 
    props=[
        ('visibility', 'hidden'),
        ('position', 'absolute'),
        ('background-color', 'white'),
        ('color', 'black'),
        ('z-index', 1),
        ('padding', '3px 3px'),
        ('margin', '2px')
    ])

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by
